<a href="https://colab.research.google.com/github/MunadHasan/BSc_Project/blob/main/ecmwf%2Bncep_forecast_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install cfgrib eccodes
!pip install cfgrib
!pip install ecmwf-api-client
!pip install netCDF4

In [ ]:
%%writefile ~/.ecmwfapirc
{
    "url"   : "https://api.ecmwf.int/v1",
    "key"   : "b9a99e6a4f44ce3ea63f63a4d1a58b52",
    "email" : "munad-1-2020017017@met.du.ac.bd"
}

In [ ]:
#@title GFS model forecast download
from ecmwfapi import ECMWFDataServer

server = ECMWFDataServer()

server.retrieve({
    "class": "ti",
    "dataset": "tigge",
    "date": "2023-03-01/to/2023-03-31",
    "expver": "prod",
    "grid": "0.5/0.5",
    "levtype": "sfc",
    "origin": "kwbc",
    "param": "228228",
    "step": "24/48/72",
    "time": "00:00:00",
    "type": "cf",
    "target": "/content/drive/MyDrive/ncep/ncep_mar_23.grib"
})

In [ ]:
#@title ECMWF forecast data download

from ecmwfapi import ECMWFDataServer

server = ECMWFDataServer()

server.retrieve({
    "class": "ti",
    "dataset": "tigge",
    "date": "2023-11-01/to/2023-11-30",
    "expver": "prod",
    "grid": "0.5/0.5",
    "levtype": "sfc",
    "origin": "ecmf",
    "param": "228228",
    "step": "24/48/72",
    "time": "00:00:00",
    "type": "cf",
    "area": "27/87/20/93",
    "target": "/content/drive/MyDrive/ECMWF/ecmwf_nov_23.grib"
})

In [ ]:
from cfgrib import open_dataset

ds = open_dataset("/content/drive/MyDrive/ECMWF/ecmwf_jun_24.grib")

nearest_coords = ds.sel(latitude=21.5, longitude=92, method='nearest')
display(nearest_coords)

selected_data = nearest_coords.sel(time='2024-06-13')
display(selected_data)

In [ ]:
multiplied_tp = selected_data['tp']

print("TP values:")
for i in range(len(multiplied_tp)):
  print(f"Step {i+1}: {multiplied_tp.isel(step=i).values}")

total_tp = multiplied_tp.sum().values
print(f"\nSum of TP values for the first 4 steps: {total_tp}")

In [ ]:
if 'tp' in ds.variables:
  tp_units = ds['tp'].attrs.get('units')
  if tp_units:
    print(f"The unit of 'tp' in ds is: {tp_units}")
  else:
    print("The 'tp' variable in ds does not have a 'units' attribute.")
else:
  print("The 'tp' variable is not found in ds.")

In [ ]:
import pandas as pd

nearest_coords_df = nearest_coords.to_dataframe()

excel_output_path = '/content/drive/MyDrive/nearest_coords.xlsx'

nearest_coords_df.to_excel(excel_output_path)

print(f"Data successfully saved to {excel_output_path}")

In [ ]:
import os

folder_path = "/content/drive/MyDrive/ECMWF"
all_files = os.listdir(folder_path)
grib_files = [f for f in all_files if f.endswith('.grib')]
print(grib_files)

In [ ]:
import xarray as xr

full_grib_paths = [os.path.join(folder_path, f) for f in grib_files]

merged_ds = xr.open_mfdataset(full_grib_paths, engine='cfgrib', combine='by_coords')

In [ ]:
output_path = "/content/drive/MyDrive/ECMWF/merged_ecmwf_data.nc"
merged_ds.to_netcdf(output_path, engine='netcdf4')
print(f"Merged dataset successfully saved to {output_path}")

In [ ]:
import os

folder_path = "/content/drive/MyDrive/ncep"
all_files = os.listdir(folder_path)
print(all_files)

In [ ]:
import xarray as xr

full_grib_paths = [os.path.join(folder_path, f) for f in all_files if f.endswith('.grib')]

merged_ncep_ds = xr.open_mfdataset(full_grib_paths, engine='cfgrib', combine='by_coords')

In [ ]:
output_path = "/content/drive/MyDrive/ncep/merged_ncep_data.nc"
merged_ncep_ds.to_netcdf(output_path, engine='netcdf4')
print(f"Merged dataset successfully saved to {output_path}")

In [ ]:
from cfgrib import open_dataset

file_path = "/content/fnl_20190227_06_00.grib2"
ds_single = open_dataset(file_path, filter_by_keys={'typeOfLevel': 'surface'})

display(ds_single)
print(ds_single.variables)